In [38]:
import pandas as pd

# Load raw timetable data
hyd_pune = pd.read_csv("timetable_hyd_pune.csv")
pune_hyd = pd.read_csv("timetable_pune_hyd.csv")
veh = pd.read_csv("vehicle_with_features.csv")

print(hyd_pune.columns)
print(pune_hyd.columns)
print(veh.columns)


Index(['SNo', 'Location', 'Primary', 'BoardingPoint', 'Time', 'Day',
       'Latitude', 'Longitude', 'PointType'],
      dtype='object')
Index(['SNo', 'Location', 'Primary', 'BoardingPoint', 'Time', 'Day',
       'Latitude', 'Longitude', 'PointType'],
      dtype='object')
Index(['Date', 'Time Range', 'Distance Travelled (km)',
       'Total Distance Travelled (km)', 'Nearest Hub', 'Address',
       'Average Speed', 'Latitude', 'Longitude', 'Altitude', 'Map Link',
       'Start Time', 'End Time', 'Timestamp', 'Speed', 'Hour', 'DayOfWeek',
       'Is_Night', 'DistanceToNearestStop_m', 'TimeDiffToStop_min',
       'IsStopped', 'StopGroup', 'StoppedDuration'],
      dtype='object')
